<a href="https://colab.research.google.com/github/BWetende/Bilha.Wetende1/blob/main/Recommender_System_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#reading google drive into google collab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#importing the dataset and necessary packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
movies = pd.read_csv('//content/drive/MyDrive/movies.dat', sep='::', header=None, names=['movie_id', 'title', 'genres'], encoding='latin-1')
ratings = pd.read_csv('/content/drive/MyDrive/ratings.dat', sep='::', engine='python', header=None, names=['userId', 'movieId', 'rating', 'timestamp'], encoding='latin-1')
users = pd.read_csv('/content/drive/MyDrive/users.dat', sep='::', engine='python', names=['UserID', 'Gender', 'Age', 'Occupation', 'Zip-code'], encoding='latin-1')

<ipython-input-148-b025bcf94501>:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies = pd.read_csv('//content/drive/MyDrive/movies.dat', sep='::', header=None, names=['movie_id', 'title', 'genres'], encoding='latin-1')


In [ ]:
movies

,movie_id,title,genres,clean_title
0,1,Toy Story (1995),Animation|Children's|Comedy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children's|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,Meet the Parents 2000
3879,3949,Requiem for a Dream (2000),Drama,Requiem for a Dream 2000
3880,3950,Tigerland (2000),Drama,Tigerland 2000
3881,3951,Two Family House (2000),Drama,Two Family House 2000


In [ ]:
users

,UserID,Gender,Age,Occupation,Zip-code
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


In [ ]:
ratings

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


In [ ]:
#Cleaning the movie titles with regex (regular expression)

import re
#movies['year'] = movies['title'].str.extract('\((\d{4})\)$', expand=True)
#movies['title'] = movies['title'].str.replace('\s*\(\d{4}\)$', '')

def clean_title(title):
    return re.sub("[^a-zA-Z0-9 ]", "", title)

#creating a new column called clean_title
movies["clean_title"] = movies["title"].apply(clean_title)
movies


,movie_id,title,genres,clean_title
0,1,Toy Story (1995),Animation|Children's|Comedy,Toy Story 1995
1,2,Jumanji (1995),Adventure|Children's|Fantasy,Jumanji 1995
2,3,Grumpier Old Men (1995),Comedy|Romance,Grumpier Old Men 1995
3,4,Waiting to Exhale (1995),Comedy|Drama,Waiting to Exhale 1995
4,5,Father of the Bride Part II (1995),Comedy,Father of the Bride Part II 1995
...,...,...,...,...
3878,3948,Meet the Parents (2000),Comedy,Meet the Parents 2000
3879,3949,Requiem for a Dream (2000),Drama,Requiem for a Dream 2000
3880,3950,Tigerland (2000),Drama,Tigerland 2000
3881,3951,Two Family House (2000),Drama,Two Family House 2000


In [ ]:
#Building a search engine
from sklearn.feature_extraction.text import TfidfVectorizer

#creating a paramenter that can search two multiple words ie Toy Story & Story 1995
vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies["clean_title"])

In [ ]:
#creating a search function ie creating a similarity between a term that we search and the movies using cosine similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Create a TfidfVectorizer to convert movie titles into vectors
vectorizer = TfidfVectorizer()

# Convert the list of movie titles to a matrix of TF-IDF features
tfidf = vectorizer.fit_transform(movies["title"].apply(clean_title))

# Create a function to search for similar movies
def search(query):
    # Clean the query string
    query = clean_title(query)

    # Convert the query string to a vector using the same vectorizer
    query_vec = vectorizer.transform([query])

    # Compute the cosine similarity between the query vector and all movie vectors
    similarity = cosine_similarity(query_vec, tfidf).flatten()

    # Sort the movies by similarity score
    sim_scores = list(enumerate(similarity))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Return the top 10 most similar movies
    top_similar_movies = [(movies.iloc[i]["title"], score) for i, score in sim_scores[:10]]
    return top_similar_movies


In [ ]:
# Define a query string
query = "Toy Story"

# Search for similar movies
results = search(query)

# Print the results
print(f"Top 10 movies similar to '{query}':")
for title, score in results:
    print(f"\t{title} ({score:.2f})")

Top 10 movies similar to 'Toy Story':
	Toy Story (1995) (0.95)
	Toy Story 2 (1999) (0.94)
	L.A. Story (1991) (0.36)
	Christmas Story, A (1983) (0.31)
	Story of Us, The (1999) (0.31)
	Soldier's Story, A (1984) (0.31)
	Lilian's Story (1995) (0.31)
	Straight Story, The (1999) (0.30)
	NeverEnding Story, The (1984) (0.30)
	Story of Xinghua, The (1993) (0.29)


In [ ]:
#Creating a search tab

import ipywidgets as widgets
from IPython.display import display

# Define the search bar widget
search_bar = widgets.Text(
    value="",
    placeholder="Enter a movie title",
    description="Search:",
    disabled=False
)

# Define the search results widget
search_results = widgets.Output()

# Define the search function
def on_search(btn):
    # Clear the previous search results
    search_results.clear_output()
    
    # Perform the search
    results = search(search_bar.value)
    
    # Display the search results
    with search_results:
        for result in results:
            print(result[0], " (score:", result[1], ")")

# Define the search button widget
search_btn = widgets.Button(
    description="Search",
    disabled=False,
    button_style="",
    tooltip="Search for movies"
)

# Attach the search function to the search button
search_btn.on_click(on_search)

# Display the search bar, search button, and search results widgets
display(widgets.HBox([search_bar, search_btn]))
display(search_results)

Output()

In [ ]:
#Creating a recommendation system
#reading the ratings dataset
ratings = pd.read_csv('/content/drive/MyDrive/ratings.dat', sep='::', header=None, names=['movie_id', 'title', 'genres'], encoding='latin-1')
ratings


<ipython-input-131-ec045051f681>:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv('/content/drive/MyDrive/ratings.dat', sep='::', header=None, names=['movie_id', 'title', 'genres'], encoding='latin-1')


,movie_id,title,genres
1,1193,5,978300760
1,661,3,978302109
1,914,3,978301968
1,3408,4,978300275
1,2355,5,978824291
...,...,...,...
6040,1091,1,956716541
6040,1094,5,956704887
6040,562,5,956704746
6040,1096,4,956715648


In [ ]:
#viewing the datatype for the ratings dataset
ratings.dtypes

movie_id    int64
title       int64
genres      int64
dtype: object

In [ ]:
#the movie id for "Toy Story" is 1
#people who liked similar movie
movie_id = 1
similar_users = ratings[(ratings["movieId"] == movie_id) & (ratings["rating"] >= 4)]["userId"].unique()
similar_users

In [179]:
#other movies that were liked by people who liked "Toy Story"
similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
similar_user_recs

0         1193
4         2355
6         1287
7         2804
10         595
          ... 
997755    1393
997758    1717
997773    2716
997790    2762
997806    1094
Name: movieId, Length: 57652, dtype: int64

In [180]:
#counts how many times each movie appears in the dataset divided by the total number of users to find %
#similar_user_recs.value_counts() / len (similar_users) 

movie_counts = similar_user_recs.value_counts()
movie_percents = movie_counts / len(similar_users)
movie_percents

1       1.000000
260     0.489024
3114    0.469512
1198    0.454878
1196    0.421951
          ...   
416     0.001220
1886    0.001220
2668    0.001220
3605    0.001220
2704    0.001220
Name: movieId, Length: 2626, dtype: float64

In [182]:
# movies that only > 10% of the similar users liked
recommended_movies = movie_percents[movie_percents > 0.1]
recommended_movies

1       1.000000
260     0.489024
3114    0.469512
1198    0.454878
1196    0.421951
          ...   
1947    0.104878
293     0.103659
2289    0.103659
377     0.101220
3897    0.101220
Name: movieId, Length: 159, dtype: float64

In [188]:
#Finding movies that most people like that are also similar to what others like most
all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4 )]
all_users

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
25,1,48,5,978824351
120,2,1193,5,978298413
157,2,920,5,978298775
231,3,1136,5,978298079
...,...,...,...,...
999890,6040,923,5,956716777
999891,6040,924,5,957717191
999933,6040,1104,5,957716904
999961,6040,1199,5,956715455


In [201]:
#Percentage of all users that recommend each of the user_all movies
all_users_count = all_users["movieId"].value_counts()
all_users_percents = all_users_count / len(all_users["userId"].unique())
all_users_percents

858     0.333786
1193    0.212039
1136    0.194388
924     0.171306
908     0.152297
          ...   
966     0.000226
1149    0.000226
2350    0.000226
860     0.000226
985     0.000226
Name: movieId, Length: 192, dtype: float64

In [205]:
rec_percentages = pd.concat([movie_percents, all_users_percents], axis = 1)
rec_percentages.columns = ["similar","all"]
rec_percentages

,similar,all
1,1.000000,NaN
260,0.489024,NaN
3114,0.469512,NaN
1198,0.454878,NaN
1196,0.421951,NaN
...,...,...
853,NaN,0.000226
2479,NaN,0.000226
966,NaN,0.000226
1149,NaN,0.000226
